# Pancancer trans effect overlap

We want to test whether the number of trans-affected proteins that we see across multiple cancer types is much different from the number we'd expect to see if we just randomly chose that proportion of proteins from each cancer type. To do this, we will use a permutation test. Each permutation will randomly select a number of proteins from each cancer type that is the same as the number that we found to be trans-affected. It will then calculate the proportions of those proteins that are common across different numbers of cancer types. This will create a null distribution for trans effect overlap among cancer types.

## Setup

In [1]:
import pandas as pd
import numpy as np
import os
import altair as alt

In [2]:
CHROMOSOME = "8"
ARM = "p"
TRANS_OR_CIS = "trans"

ttest_results_file = f"{CHROMOSOME}{ARM}_{TRANS_OR_CIS}effects_ttest.tsv"

## Load and parse trans effects file

In [3]:
ttest_results = pd.\
read_csv(ttest_results_file, sep="\t").\
rename(columns={"Name": "protein"}).\
set_index("protein")

cancer_types = sorted(ttest_results.columns.to_series().str.split("_", n=1, expand=True)[0].unique())

long_results = pd.DataFrame()

for cancer_type in cancer_types:
    cancer_df = ttest_results.\
    loc[:, ttest_results.columns.str.startswith(cancer_type)].\
    dropna(axis="index", how="all").\
    reset_index(drop=False)
    
    # If the cancer type has database IDs, make a separate column that has them.
    # If not, create a column of NaNs (so that the tables all match)
    if f"{cancer_type}_Database_ID" in cancer_df.columns:
        cancer_df = cancer_df.rename(columns={f"{cancer_type}_Database_ID": "Database_ID"})
    else:
        cancer_df = cancer_df.assign(Database_ID=np.nan)
        
    # Rename the pvalue and diff columns to not have the cancer type
    cancer_df = cancer_df.rename(columns={
        f"{cancer_type}_pvalue": "adj_p",
        f"{cancer_type}_diff": "change"
    }).\
    assign(cancer_type=cancer_type)
    
    # Reorder the columns
    cancer_df = cancer_df[["cancer_type", "protein", "Database_ID", "adj_p", "change"]]
    
    # Append to the overall dataframe
    long_results = long_results.append(cancer_df)

# Drop duplicate rows and reset the index
long_results = long_results[~long_results.duplicated(keep=False)].\
reset_index(drop=True)

## Calculate the proportion of proteins that were significantly trans-affected in each cancer type

In [4]:
props_different = long_results.\
groupby("cancer_type")["adj_p"].\
apply(lambda x: (x <= 0.05).sum() / (x > 0.05).sum())

props_different

cancer_type
brca       0.000111
colon      0.001793
hnscc      0.001481
lscc       0.000000
luad       0.004726
ovarian    0.000000
Name: adj_p, dtype: float64